In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_FOLDER = "/content/drive/MyDrive/Datasets/images/images"

## Imports Libraries

In [ ]:
import torch
import torchvision
from torchvision import transforms, utils
import torch.optim as optim

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

1.7.0+cu101
0.8.1+cu101


## ImageFolder

In [ ]:
datasetImageFolder = torchvision.datasets.ImageFolder(DATASET_FOLDER, transform=transforms.Compose([
                                               transforms.Resize((299, 299)),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                                                     
]))

In [ ]:
print(datasetImageFolder.classes)
print('total classes', len(datasetImageFolder.classes))
print('total iamges', len(datasetImageFolder))

['Albrecht_DuÔòá├¬rer', 'Albrecht_Du╠êrer', 'Alfred_Sisley', 'Amedeo_Modigliani', 'Andrei_Rublev', 'Andy_Warhol', 'Camille_Pissarro', 'Caravaggio', 'Claude_Monet', 'Diego_Rivera', 'Diego_Velazquez', 'Edgar_Degas', 'Edouard_Manet', 'Edvard_Munch', 'El_Greco', 'Eugene_Delacroix', 'Francisco_Goya', 'Frida_Kahlo', 'Georges_Seurat', 'Giotto_di_Bondone', 'Gustav_Klimt', 'Gustave_Courbet', 'Henri_Matisse', 'Henri_Rousseau', 'Henri_de_Toulouse-Lautrec', 'Hieronymus_Bosch', 'Jackson_Pollock', 'Jan_van_Eyck', 'Joan_Miro', 'Kazimir_Malevich', 'Leonardo_da_Vinci', 'Marc_Chagall', 'Michelangelo', 'Mikhail_Vrubel', 'Pablo_Picasso', 'Paul_Cezanne', 'Paul_Gauguin', 'Paul_Klee', 'Peter_Paul_Rubens', 'Pierre-Auguste_Renoir', 'Piet_Mondrian', 'Pieter_Bruegel', 'Raphael', 'Rembrandt', 'Rene_Magritte', 'Salvador_Dali', 'Sandro_Botticelli', 'Titian', 'Vasiliy_Kandinskiy', 'Vincent_van_Gogh', 'William_Turner']
total classes 51
total iamges 8785


In [ ]:
train_size = round(0.9 * len(datasetImageFolder))
test_size = len(datasetImageFolder) - train_size
print(train_size, test_size)

7906 879


In [ ]:
batch_size = 32
train_dataset, test_dataset = torch.utils.data.random_split(datasetImageFolder, [train_size, test_size])

traindl = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
testdl = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)

#Googlenet Model

In [ ]:
googlenet = torchvision.models.googlenet(pretrained=True, progress=True)
googlenet.fc = torch.nn.Linear(1024, 51)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [ ]:
device = torch.device('cuda')
googlenet = googlenet.to(device)

In [ ]:
print(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(googlenet.parameters(), lr=0.001, momentum=0.9)

#Training

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(traindl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = googlenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')


#Pytorch model

In [ ]:
ONNX_PATH="./meraArtistWalaModel.onnx"
TF_PATH = "./meraArtistWalaModel.pb" # where the representation of tensorflow model will be stored
TFLITE_PATH = "./meraArtistWalaModel.tflite"

In [ ]:
PATH = './artistifyModel.pt'
torch.save(googlenet.state_dict(), PATH)

#ONNX model

In [ ]:
import onnx
import torch

example_input = torch.random((1, 3, 299, 299))
 # exmample for the forward pass input 
pytorch_model = googlenet

torch.onnx.export(
    model=pytorch_model,
    args=example_input, 
    f=ONNX_PATH, # where should it be saved
    verbose=False,
    export_params=True,
    do_constant_folding=False,  # fold constant values for optimization
    # do_constant_folding=True,   # fold constant values for optimization
    input_names=['input'],
    output_names=['output']
)
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

#Tensorflow model

In [ ]:
from onnx_tf.backend import prepare
import onnx


onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

#Tensorflowlite model

In [ ]:

# protopuf needs your virtual environment to be explictly exported in the path
os.environ["PATH"] = "/opt/miniconda3/envs/convert/bin:/opt/miniconda3/bin:/usr/local/sbin:...."

# make a converter object from the saved tensorflow file
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(TF_PATH,  # TensorFlow freezegraph .pb model file
                                                      input_arrays=['input'], # name of input arrays as defined in torch.onnx.export function before.
                                                      output_arrays=['output'] # name of output arrays defined in torch.onnx.export function before.
                                                      )

# tell converter which type of optimization techniques to use
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]

converter.experimental_new_converter = True

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)